Instalamos la libreria:
* Sentence transformers: https://www.sbert.net/

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Importamos las diferentes librerias:

In [10]:
from sentence_transformers import SentenceTransformer, util
import json
from google.colab import drive
import numpy as np
import time

Debido a que tenemos frases cortas, y perfectamente diferenciadas, usamos un modelo ligero, y lo ejecutamos en CPU:
* https://huggingface.co/intfloat/multilingual-e5-small

Parámetros del modelo:
* Memoria: 471 mb
* Input length: 512 tokens
* Output size embeddings: 384

In [3]:
model = SentenceTransformer('intfloat/multilingual-e5-small')
print(model)
passage = []
doc_emb = None
DEVICE = "cpu"
data = None
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Mounted at /content/drive


Definición de las funciones:
* **loadJson(filename)**: Carga el contenido del json, y codifica con el modelo el campo *description* de cada objeto.

* **calling_function**: Codifica la consulta introducida, y realiza una comparación con la similitud del coseno con el resto de vectores, obteniendo el indice del objeto con el campo *description* más similiar. Previamente, se define un umbral (0.85) para determinar que el score obtenido en la comparación es fiable. Una vez obtenido este objeto, se ejecuta la función asociada en el campo *function*.

In [13]:
def loadJson(fileName= None):
    if fileName is None:
        fileName = 'actions.json'
    with open(fileName, encoding='utf-8') as f:
        data = json.load(f)

    passage = []
    for element in data:
        passage.append("passage: " + element["description"])
    print("Processing items: ",len(passage))
    start = time.process_time()
    doc_emb = model.encode(passage, normalize_embeddings=True, show_progress_bar=True, device=DEVICE, batch_size=16)
    end = time.process_time()
    print("Processing time:",end - start)
    return doc_emb, data

def calling_function(text):
    #Encode query
    query_emb = model.encode("query: " + text, normalize_embeddings=True, show_progress_bar=True, device=DEVICE, batch_size=16)
    #Compute dot score between query and all document embeddings
    scores = util.cos_sim(query_emb, doc_emb)[0]
    #print(scores.argmax())
    #print(data[scores.argmax()])
    #scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    #Combine docs & scores
    #doc_score_pairs = list(zip(data, scores.tolist()))
    #Sort by decreasing score
    #doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    #for doc, score in doc_score_pairs[:5]:
        #print(score, doc)
    #    print("(Score: {:.4f}) Function: {}, Description: {}".format(score,doc["function"], doc["description"]) )
    if scores.max() > 0.85:
      function_execute = data[scores.argmax()]["function"]
      eval(function_execute+"()")
    else:
      print("No se ha encontrado una función similar.")

Se definen las funciones necesarias de nuestro sistema.

In [8]:
def action_show_holidays():
  print("Mis vacaciones son: ...")

def action_how_horasSindicales():
  print("Mis horas sindicales son: ...")

def action_show_signings():
  print("Mis fichajes son: ...")

Se carga el fichero de configuración *actions.json* y se codifican los campos *description* de cada objeto.

---



In [14]:
#%cd /content/drive/My Drive/UOC/Docs/
#!ls "/content/drive/My Drive/UOC/Docs"
doc_emb, data = loadJson("/content/drive/My Drive/UOC/Docs/actions.json")


Processing items:  7


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing time: 0.24886956699999985


Se procede a llamar a la función que más se asimila a la consulta realizada. En caso de que no haya un resultado similar dentro del umbral preestablecido, no ejecuta nada.

In [16]:
query = "¿Cuantas horas sindicales tengo?"
calling_function(query)

query = "¿Qué pasa con el tiempo?"
calling_function(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Mis horas sindicales son: ...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No se ha encontrado una función similar.
